In [5]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [7]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [530]:
period = '_MarchApril2021'  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [531]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [532]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  1376
USA:  3957
China&USA:  5333


In [533]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 1376
1000 / 1376
0 / 3957
1000 / 3957
2000 / 3957
3000 / 3957
0 / 5333
1000 / 5333
2000 / 5333
3000 / 5333
4000 / 5333
5000 / 5333


In [534]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['world health organization express concern rise rate new covid infection death third wave virus sweep across world', 'director general world health organization warn rise global covid case death press conference emphasize public follow covid health guideline contain virus', 'vast majority covid vaccine administer far wealthy nation world health organization report', 'africa administer percent million covid vaccine dos give globally world health organization say', 'causal link covid vaccine rare occurrence blood clot low platelet consider plausible confirm world health organization say', 'mark annual world health day year call fairer healthy world everyone eliminate health inequity pandemic push people poverty food insecurity social problem', 'covid19 vaccine produce expect review emergency use end say accord people daily', 'region currently fight resurgence covid transmission world health organization regional office call upon government region scale vaccine production vaccinat

In [541]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  2005
USA:  3334
China&USA:  3860


In [544]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>50])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>150])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>200])

China
['covid', 'vaccine', 'china', 'say', 'report', 'health', 'case', 'country', 'receive', 'first', 'million', 'new', 'dos', 'people', 'pandemic', 'batch', 'world', 'president', 'accord', 'dose', 'use', 'national', 'death', 'day', 'vaccination', 'one', 'year', 'brazil', 'administer', 'second', 'arrive', 'state', 'minister', 'city']

USA
['covid', 'vaccine', 'say', 'new', 'case', 'johnson', 'shot', 'million', 'report', 'use', 'country', 'people', 'get', 'india', 'health', 'first', 'dos', 'blood', 'vaccination', 'clot', 'pandemic', 'coronavirus', 'death', 'state', 'year', 'brazil', 'surge']

China&USA
['covid', 'vaccine', 'say', 'case', 'new', 'china', 'report', 'country', 'million', 'health', 'first', 'people', 'johnson', 'dos', 'pandemic', 'use', 'shot', 'receive', 'get', 'vaccination', 'death', 'state', 'blood', 'coronavirus', 'india', 'clot', 'year', 'world', 'brazil', 'president', 'day']


# less frequent words

In [546]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<2]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=2]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<2]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<2]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=2]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<2]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<2]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=2]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<2]

Less frequent China:  777
More frequent China:  1228
Less frequent USA:  916
More frequent USA: 2418
Less frequent China&USA:  1123
More frequent China&USA:  2737


In [547]:
# discard less frequent words
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [548]:
cleaned_mostfreq_text_China = clean_words(cleaned_text_China, less_frequent_words_China)
cleaned_mostfreq_text_USA = clean_words(cleaned_text_USA, less_frequent_words_USA)
cleaned_mostfreq_text_China_USA = clean_words(cleaned_text_China_USA, less_frequent_words_China_USA)

0 / 1336
0 / 3957
2000 / 3957
0 / 5293
2000 / 5293
4000 / 5293


In [549]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  1228
USA:  2418
China&USA:  2737


# Build Network

In [550]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    959
1       vaccine    767
2         china    329
3           say    191
4        report    175
5        health    173
6          case    172
7       country    172
8       receive    159
9         first    156
10      million    151
11          new    150
12          dos    123
13       people    120
14     pandemic    113
15        batch    105
16        world     96
17    president     77
18       accord     72
19         dose     71
20          use     70
21     national     67
22        death     65
23          day     65
24  vaccination     65
25          one     64
26         year     63
27       brazil     63
28   administer     61
29       second     59

USA
           Word  Count
0         covid   3021
1       vaccine   2128
2           say    879
3           new    413
4          case    395
5       johnson    298
6          shot    259
7       million    258
8        report    253
9           use    253
10      country    247


In [551]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [518]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [552]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [553]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       529
(vaccine, china)       254
(china, covid)         250
(receive, vaccine)     171
(covid, case)          143
(million, covid)       141
(health, covid)        139
(vaccine, say)         137
(vaccine, dos)         135
(covid, say)           135
(country, vaccine)     134
(covid, country)       127
(first, vaccine)       125
(new, covid)           125
(covid, report)        121
(first, covid)         119
(receive, covid)       118
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(covid, dos)           105
(covid, pandemic)      103
(new, case)             98
(covid, people)         92
(batch, covid)          90
(million, dos)          90
(vaccine, use)          83
(case, report)          81
(world, covid)          74
(dose, vaccine)         73

USA:
                      weight
(vaccine, covid)        1459
(covid, say)             649
(vaccine, say)           633
(new, covid)             363
(covi

#Graph


In [521]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [554]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [555]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  1228
Edges:  32532
Is connected:  True

USA:
Nodes:  2418
Edges:  87358
Is connected:  True

China&USA:
Nodes:  2737
Edges:  113899
Is connected:  True


# PageRank

In [556]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [525]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [526]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [557]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  523

              0         1
0         covid  0.061217
1       vaccine  0.047503
2         china  0.022432
3           say  0.015631
4        health  0.014202
5       country  0.013810
6        report  0.012769
7          case  0.011512
8         first  0.010615
9       million  0.010405
10      receive  0.009780
11          new  0.009638
12       people  0.009204
13          dos  0.008638
14        world  0.007912
15     pandemic  0.007704
16       accord  0.005808
17        batch  0.005489
18          day  0.005189
19     national  0.005167
20          use  0.005116
21    president  0.005032
22        death  0.004961
23  vaccination  0.004882
24         year  0.004831
25       brazil  0.004695
26         dose  0.004447
27   administer  0.004420
28       origin  0.004385
29         city  0.004313

USA:  521

              0         1
0         covid  0.058642
1       vaccine  0.042766
2           say  0.021170
3           new  0.008835
4          case  0.008359
5       johns

# reduced graph

In [558]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [559]:
cleaned_mostimp_text_China = clean_words(cleaned_mostfreq_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_mostfreq_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_mostfreq_text_China_USA,less_important_words_China_USA)

0 / 1336
0 / 3957
2000 / 3957
0 / 5293
2000 / 5293
4000 / 5293


In [560]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  523
USA:  521
China&USA:  509


In [561]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    959
1       vaccine    767
2         china    329
3           say    191
4        report    175
5        health    173
6          case    172
7       country    172
8       receive    159
9         first    156
10      million    151
11          new    150
12          dos    123
13       people    120
14     pandemic    113
15        batch    105
16        world     96
17    president     77
18       accord     72
19         dose     71
20          use     70
21     national     67
22        death     65
23          day     65
24  vaccination     65
25          one     64
26         year     63
27       brazil     63
28   administer     61
29       second     59

USA
           Word  Count
0         covid   3021
1       vaccine   2128
2           say    879
3           new    413
4          case    395
5       johnson    298
6          shot    259
7       million    258
8        report    253
9           use    253
10      country    247


In [562]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [563]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [564]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       529
(vaccine, china)       254
(china, covid)         250
(receive, vaccine)     171
(covid, case)          143
(million, covid)       141
(health, covid)        139
(vaccine, say)         137
(vaccine, dos)         135
(covid, say)           135
(country, vaccine)     134
(covid, country)       127
(first, vaccine)       125
(new, covid)           125
(covid, report)        121
(first, covid)         119
(receive, covid)       118
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(covid, dos)           105
(covid, pandemic)      103
(new, case)             98
(covid, people)         92
(batch, covid)          90
(million, dos)          90
(vaccine, use)          83
(case, report)          81
(world, covid)          74
(dose, vaccine)         73

USA:
                      weight
(vaccine, covid)        1459
(covid, say)             649
(vaccine, say)           633
(new, covid)             363
(covi

In [565]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [566]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [567]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  523
Edges:  18792
Is connected:  True

USA:
Nodes:  521
Edges:  34979
Is connected:  True

China&USA:
Nodes:  509
Edges:  40744
Is connected:  True


#Save edge list

In [568]:
filename = './edgelist_China_MarchApril2021.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_MarchApril2021.csv
files.download("edgelist_China_MarchApril2021.csv")

filename = './edgelist_USA_MarchApril2021.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_MarchApril2021.csv
files.download("edgelist_USA_MarchApril2021.csv")

filename = './edgelist_China_USA_MarchApril2021.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_MarchApril2021.csv
files.download("edgelist_China_USA_MarchApril2021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [570]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [569]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_MarchApril2021")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_MarchApril2021")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA_MarchApril2021")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2    china    china
3      say      say
4   report   report

USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3      new      new
4     case     case

China&USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3     case     case
4      new      new

